In [271]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [272]:
%cd '/content/drive/My Drive/RL/amalearn'

/content/drive/My Drive/RL/amalearn


In [273]:
!pwd

/content/drive/My Drive/RL/amalearn


In [274]:
!pip install -e .

Obtaining file:///content/drive/My%20Drive/RL/amalearn
  Attempting uninstall: amalearn
    Found existing installation: amalearn 0.1
    Can't uninstall 'amalearn'. No files were found to uninstall.
  Running setup.py develop for amalearn


In [275]:
from amalearn.reward import RewardBase
from amalearn.agent import AgentBase
from amalearn.environment import EnvironmentBase
import gym
import numpy as np
import random

In [276]:
grid_size = 15
action_size = 9
list_of_actions = [[0,0], [0,1], [0,-1], [1,0], [-1,0],[-1,1], [-1,-1], [1,-1], [1,1]]
obstacle = [[0,6],[0,7],[1,6],[1,7],[2,6],[2,7],[3,6],[3,7],[7,12],[7,13],[7,14],[8,12],[8,13],[8,14],[11,5],[11,6],[12,5],[12,6],[13,5],[13,6],[14,5],[14,6]]
goal = [14, 14]
# initial_state = [14, 14]

In [277]:
class Environment(EnvironmentBase):
    def __init__(self, obstacle, id, action_count, actionPrice, goalReward, punish, j_limit, i_limit, p, container):
      self.obstacle = obstacle
      self.action_count= action_count
      self.actionPrice = actionPrice
      self.goalReward = goalReward
      self.punish= punish
      self.j_limit = j_limit
      self.i_limit = i_limit
      self.p = p
      self.container = container
  
    def isStatePossible(self, state):
      for i in range(len(self.obstacle)):
        if state == self.obstacle[i]:
          return False
        elif not ((-1 < state[0] < 15) and (-1 < state[1] <15)):
          return False
        else: 
          return True

    def isAccessible(self, state, state_p):
     
      for action in list_of_actions:
        if [sum(x) for x in zip(state, action)] == state_p:
          if state_p not in self.obstacle:
            return True
          else:
            return False
        else:
           return False 
        
    
    def getTransitionStatesAndProbs(self, state, action, state_p):

      prob = np.zeros(9)
      action_index = list_of_actions.index(action)
      if action_index in self.available_actions(state):
        n = len(self.available_actions(state))-1
        if n !=0:
          prob[action_index] = self.p
          for i in self.available_actions(state):
            if i != action_index:
          # print(self.available_actions(state))
              prob[i] = (1-self.p)/n
          prob_stay=(1-self.p)/n
        else:
          prob[action_index] = self.p
          prob[0] = 1-self.p
          prob_stay = 1-self.p
      else:
        prob[0] = self.p
        n = len(self.available_actions(state))
        if n !=0:
          prob[action_index] = 0
          for i in self.available_actions(state):
            if i != action_index:
          # print(self.available_actions(state))
              prob[i] = (1-self.p)/n
        prob_stay = (1-self.p)/n
      return prob,prob_stay
      

    def getReward(self, state, action, state_p):

      if (state_p[0] == state[0]) and (state_p[1] == state[1]):
        return self.actionPrice
      elif [state_p[0],state_p[1]] in self.obstacle:
        return self.actionPrice + self.punish
      elif (not([state_p[0],state_p[1]] in self.obstacle)) and ((state_p[0] != goal[0]) or state_p[1] != goal[1]):
        return self.actionPrice
      elif (state_p[0] == goal[0]) and (state_p[1] == goal[1]):
        return self.goalReward + self.actionPrice


    def getReward2(self, state, action, state_p):

      if (state_p[0] == state[0]) and (state_p[1] == state[1]):
        return 0
      elif [state_p[0],state_p[1]] in self.obstacle:
        return self.actionPrice + self.punish
      elif (not([state_p[0],state_p[1]] in self.obstacle)) and ((state_p[0] != goal[0]) or state_p[1] != goal[1]):
        return self.actionPrice
      elif (state_p[0] == goal[0]) and (state_p[1] == goal[1]):
        return self.goalReward + self.actionPrice   
    
    def sample_all_rewards(self):
        return 
    
    def calculate_reward(self, action):
        return 

    def terminated(self):
        return 

    def observe(self):
        return 

    def available_actions(self, state):
      available__actions = []
      available__actions = list_of_actions.copy()
      for action in list_of_actions:
        if [[sum(x) for x in zip(state, action)][0],[sum(x) for x in zip(state, action)][1]] in self.obstacle:
          available__actions.remove(action)
        if ([sum(x) for x in zip(state, action)][0] < 0) or ([sum(x) for x in zip(state, action)][1] < 0) or ([sum(x) for x in zip(state, action)][0] > 14) or ([sum(x) for x in zip(state, action)][1] > 14):
          available__actions.remove(action)
      
      index_of_availables = []
      for i in range(9):
        for j in range(len(available__actions)):
          if list_of_actions[i] == available__actions[j]:
            index_of_availables.append(i)
      return index_of_availables

    def next_state(self, action):
        return

    def reset(self):
        return

    def render(self, mode='human'):
        #print('{}:\taction={}'.format(self.state['length'], self.state['last_action']))
        return 

    def close(self):
        return

In [278]:
import numpy as np

class Agent(AgentBase):
    def __init__(self, id, environment, discount, theta, value, policy, map):
        self.environment = environment
        self.mapp = map
        #mapp states to its ids
        self.V = value
        self.policy = policy
        super(Agent, self).__init__(id, environment)
        self.discount = discount
        self.theta = theta
        self.policy_stable = None
        


    def bellman_equation_action(self, state, action):
      returns = 0
      for s_p in range(len(list_of_actions)):
        if s_p in self.environment.available_actions(state):
      
          next_state = [sum(x) for x in zip(state, list_of_actions[s_p])]
          probs,_ = self.environment.getTransitionStatesAndProbs(state, list_of_actions[action], next_state)
          
          reward = self.environment.getReward(state, action, next_state)
          returns += 1 * probs[s_p] * (reward + self.discount * self.V[int(next_state[0]),int(next_state[1])])

        else: 
          next_state = state
          probs,prob_stay= self.environment.getTransitionStatesAndProbs(state, list_of_actions[action], next_state)
          next_state = obstacle[s_p]
          reward = self.environment.getReward(state, action, next_state)
          next_state = state
          returns += 1 * prob_stay * (reward + self.discount * self.V[int(next_state[0]),int(next_state[1])])

      return returns
    
    def bellman_equation_action2(self, state, action):
      returns = 0
      if action in self.environment.available_actions(state):
      
          next_state = [sum(x) for x in zip(state, list_of_actions[action])]
          probs,_ = self.environment.getTransitionStatesAndProbs(state, list_of_actions[action], next_state)
          
          reward = self.environment.getReward(state, action, next_state)
          returns += 1 * probs[action] * (reward + self.discount * self.V[int(next_state[0]),int(next_state[1])])
          
      else: 
          next_state = state
          probs,prob_stay= self.environment.getTransitionStatesAndProbs(state, list_of_actions[action], next_state)
          next_state = obstacle[action]
          reward = self.environment.getReward(state, action, next_state)
          next_state = state
          returns += 1 * prob_stay * (reward + self.discount * self.V[int(next_state[0]),int(next_state[1])])

      return returns

    def bellman_equation_action3(self, state, action):
      returns = []

      for s_p in range(len(list_of_actions)):
        if s_p in self.environment.available_actions(state):
      
          next_state = [sum(x) for x in zip(state, list_of_actions[s_p])]
          probs,_ = self.environment.getTransitionStatesAndProbs(state, list_of_actions[action], next_state)
          
          reward = self.environment.getReward(state, action, next_state)
          returns.append(1 * probs[s_p] * (reward + self.discount * self.V[int(next_state[0]),int(next_state[1])]))

        else: 
          next_state = state
          probs,prob_stay= self.environment.getTransitionStatesAndProbs(state, list_of_actions[action], next_state)
          next_state = obstacle[s_p]
          reward = self.environment.getReward(state, action, next_state)
          next_state = state
          returns.append(1 * prob_stay * (reward + self.discount * self.V[int(next_state[0]),int(next_state[1])]))

      
      returns = np.array(returns)
      max_element = np.amax(returns)
      p = np.argmax(returns)
      print(max_element, p)
      return max_element, p

    def policy_evaluation(self):
      print('policy evaluation ...')
      while True:
        old_value = self.V.copy()
        for i in range(grid_size):
          for j in range(grid_size):
            new_state_value = self.bellman_equation_action2([i,j], self.policy[i,j])
            self.V[i, j] = new_state_value
        max_value_change = abs(old_value - self.V).max()
        if max_value_change < self.theta:
          break

    def policy_improvement(self):
      print('policy improvement ...')
      self.policy_stable = True
      for i in range(grid_size):
        for j in range(grid_size):
          old_action = self.policy[i, j]
          action_returns = []
          for action in list_of_actions:
            # if action in self.environment.available_actions([i,j]):
              action_returns.append(self.bellman_equation_action([i,j], list_of_actions.index(action)))
            # else:
              # action_returns.append(-np.inf)
          # print("action_returns",action_returns)
          new_action = list_of_actions[np.argmax(action_returns)]
          # print("new_action",new_action)
          # print("action_index",list_of_actions.index(new_action))

          for p in range(len(list_of_actions)):
            if list_of_actions[p] == new_action:
              new_action_index = p

          self.policy[i, j] = new_action_index
          print(self.policy)
          # print(self.V)
          if self.policy_stable and old_action != new_action_index:
            self.policy_stable = False
      print(' *policy_stable: {}*'.format(self.policy_stable))
      return self.policy_stable,self.policy
    
    
    def policy_iteration(self):
      while True:
        # print(self.policy)
        self.policy_evaluation()
        a,b=self.policy_improvement()
        print(b)
        self.policy = b
        if self.policy_stable:
            break
            print(self.policy)
      print('Done!')

    def value_iteration(self):
      print('value iteration ...')
      while True:
        old_value = self.V.copy()
        for i in range(15):
          for j in range(15):
            new_state_value,p = self.bellman_equation_action3([i,j], self.policy[i,j])
            self.V[i, j] = new_state_value
            self.policy = p
        max_value_change = abs(old_value - self.V).max()
        if max_value_change < self.theta:
          break
      
      return self.V[i, j], self.policy
  
    def take_action(self,state,action):# -> (object, float, bool, object):
      state_p = []
      a = list_of_actions[action]
      state_p = [sum(x) for x in zip(state, a)]
      return state_p

In [279]:
env = Environment(obstacle = obstacle, id='1',action_count=9, actionPrice = -1, goalReward = 100, punish=-10, j_limit = 15, i_limit = 15, p = 0.8, container = None)
id = '1'
theta = 0.001
discount = 0.9
value = np.zeros((grid_size,grid_size))
policy = np.zeros(value.shape, dtype=np.int)
# policy = np.eye(grid_size, dtype=np.int)

policy[0,:] = 5
policy[1,:] = 1
policy[2,:] = 8
policy[3,:] = 3
policy[4,:] = 0
policy[5,:] = 7
policy[6,:] = 7
policy[7,:] = 8
policy[8,:] = 2
policy[9,:] = 3
policy[10,:] = 8
policy[11,:] = 4
policy[12,:] = 1
policy[13,:] = 2
policy[14,:] = 4

agent1 = Agent(id, env, discount, theta, value, policy, map)

In [280]:
print(agent1.policy_iteration())
# print('expected return : {}'.format(enviroment.returns))
# print(agent1.value_iteration())

Streaming output truncated to the last 5000 lines.
 [2 8 3 3 3 3 0 0 3 3 3 3 3 3 3]
 [2 8 8 3 3 7 0 0 8 3 3 3 3 3 7]
 [2 5 1 8 3 3 3 3 3 3 3 3 3 3 3]
 [2 5 1 8 8 3 3 3 3 3 3 3 3 3 7]
 [2 5 1 5 1 8 3 3 3 3 3 3 3 7 7]
 [2 5 1 5 1 1 8 3 3 3 3 3 7 2 2]
 [2 5 1 5 1 1 1 8 3 3 3 3 7 6 0]
 [2 5 1 5 1 1 1 1 8 3 3 3 8 3 7]
 [2 5 1 5 1 1 1 1 1 8 3 3 3 7 7]
 [2 5 1 5 1 1 5 5 1 1 8 3 3 3 3]
 [2 5 1 5 5 5 1 1 1 1 8 8 8 7 3]
 [2 5 1 5 4 6 5 8 1 1 8 8 8 3 3]
 [2 5 5 4 4 0 5 8 1 1 8 1 1 8 3]
 [2 5 4 4 1 0 1 1 1 5 1 1 1 1 2]]
[[2 4 4 4 4 1 0 0 2 4 4 4 4 4 1]
 [2 8 3 3 3 3 0 0 3 3 3 3 3 3 3]
 [2 8 8 3 3 7 0 0 8 3 3 3 3 3 7]
 [2 5 1 8 3 3 3 3 3 3 3 3 3 3 3]
 [2 5 1 8 8 3 3 3 3 3 3 3 3 3 7]
 [2 5 1 5 1 8 3 3 3 3 3 3 3 7 7]
 [2 5 1 5 1 1 8 3 3 3 3 3 7 2 2]
 [2 5 1 5 1 1 1 8 3 3 3 3 7 6 0]
 [2 5 1 5 1 1 1 1 8 3 3 3 8 3 7]
 [2 5 1 5 1 1 1 1 1 8 3 3 3 7 7]
 [2 5 1 5 1 1 5 5 1 1 8 3 3 3 3]
 [2 5 1 5 5 5 1 1 1 1 8 8 8 7 3]
 [2 5 1 5 4 6 5 8 1 1 8 8 8 3 3]
 [2 5 5 4 4 0 5 8 1 1 8 1 1 8 3]
 [2 5 4 4 1 0 1 1 1 5 1 